# Cleaned data prep code

### The following code works as it is supposed to, but it can't complete test_train_split because (I think) not all the NaN-filled columns have been addressed.  

#### Code:

In [16]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@authors: 
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Function to drop weight columns
def drop_weights(data_frame):
    coltodrop = []
    for i in range(1, 81):
        coltodrop.append('WGTP' + str(i))
    return data_frame.drop(coltodrop, axis = 1)

# Function to drop allocation columns
def drop_allocations(data_frame):
    col = data_frame.columns
    coltodrop = []
    for c in col:
        if (c[0]== 'F') & (c[-1] == 'P') & \
           (c != 'FULFP') & (c != 'FULP') & (c != 'FINCP'): 
            coltodrop.append(c)
    return data_frame.drop(coltodrop, axis = 1)

# Function to recode the categorical features 
def recode(data_frame):
    # Binary categories
    dict_bin = {0: 1, 1: 2}
    bin_cols = ['HUGCL', 'NPP', 'NR', 'PSF', 'R18']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    dict_bin = {1: 2, 2: 1}
    bin_cols = ['BROADBND', 'COMPOTHX', 'DIALUP', 'HISPEED', 'LAPTOP', 'OTHSVCEX', \
                'SATELLITE', 'SMARTPHONE', 'TABLET', 'TEL', 'BATH', 'REFR', \
                'SINK', 'STOV', 'KIT', 'RNTM']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    dict_bin = {0: 2, 1: 1}
    bin_cols = ['SRNT', 'SVAL']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    # Three categories
    dict_bin = {1: 2, 2: 1, 9:0}
    bin_cols = ['RWAT', 'RWATPR', 'PLM', 'PLMPRP', 'HOTWAT']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    dict_bin = {1: 2, 2: 1, 3: 3}
    bin_cols = ['ELEFP', 'FULFP', 'GASFP', 'WATFP']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    return data_frame
    
def recode_y(data_frame):
    dict_bin = {1: 2, 2: 1}
    data_frame = data_frame.map(dict_bin)
    
# Imputer
def imputer(data_frame):
    data_frame = data_frame.dropna(subset=['HINCP'])
    data_frame.loc[:,'HINCP'] = data_frame.HINCP * \
                          data_frame.ADJINC / (10**6) 
    data_frame['WIF'].fillna(4, inplace=True)
    data_frame['WORKSTAT'].fillna(16, inplace=True)
    zero_list = ['VEH', 'BATH', 'REFR', 'SINK', 'STOV', 'KIT', 'BDSP', 'BLD', 'RWAT', 'RWATPR', 'PLM', 'PLMPRP', 'SRNT', 'SVAL', 'MV', 'TEN', 'CONP', 'ELEFP', 'ELEP', 'FULFP', 'FULP', 'GASFP', 'GASP', 'HFL']
    for col in zero_list:
        data_frame[col].fillna(0, inplace=True)
    data_frame = data_frame.drop(['ACR', 'AGS', 'FINCP', 'RESMODE', 'VALP', 'VACS'], axis = 1)

    return data_frame    
    
# Import the nationwide data-it is presented in two files 
file_dir = '/Users/flatironschol/FIS-Projects/Module5/data/'
df_a = pd.read_csv('psam_husa.csv') #replace with f'{file_dir}psam_husa.csv if necessary
df_b = pd.read_csv('psam_husb.csv') #replace with f'{file_dir}psam_husb.csv if necessary
df_a = drop_weights(df_a)
df_a = drop_allocations(df_a)
df_b = drop_weights(df_b)
df_b = drop_allocations(df_b)
df = pd.concat((df_a, df_b), axis = 0, ignore_index = True) 
y = df.FS
X = df.drop('FS', axis = 1)
X = recode(X)
y = recode_y(y)
X = imputer(X)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, \
#                                                     stratify = y, random_state = 1007)

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
//anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


#### The columns with NaN's are (along with the NaN's present in y/df.FS):

In [155]:
X.columns[X.isna().any()].tolist()

['BROADBND',
 'DIALUP',
 'HISPEED',
 'INSP',
 'MHP',
 'MRGI',
 'MRGP',
 'MRGT',
 'MRGX',
 'OTHSVCEX',
 'RNTM',
 'RNTP',
 'SATELLITE',
 'SMP',
 'WATP',
 'FES',
 'FPARC',
 'GRNTP',
 'GRPIP',
 'NPF',
 'OCPIP',
 'SMOCP',
 'SMX',
 'TAXAMT',
 'WKEXREL']

## -----anything below this is just a scratch pad thing-----

In [17]:
tester = df

In [22]:
tester = tester.dropna(axis = 1)

In [29]:
tester = tester.drop(columns = ['RT', 'SERIALNO'])

In [30]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [34]:
def calculate_vif(X, thresh=100):
    cols = X.columns
    variables = np.arange(X.shape[1])
    dropped=True
    while dropped:
        dropped=False
        print("dropped")
        c = X[cols[variables]].values
        vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]
        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
            variables = np.delete(variables, maxloc)
            dropped=True
    print('Remaining variables:')
    print(X.columns[variables])
    return X[cols[variables]]

In [36]:
calculate_vif(tester)

dropped
Remaining variables:
Index(['DIVISION', 'PUMA', 'REGION', 'ST', 'ADJHSG', 'ADJINC', 'WGTP', 'NP',
       'TYPE'],
      dtype='object')


,DIVISION,PUMA,REGION,ST,ADJHSG,ADJINC,WGTP,NP,TYPE
0,6,1600,3,1,1000000,1013097,0,1,3
1,6,1900,3,1,1000000,1013097,0,1,3
2,6,2000,3,1,1000000,1013097,0,1,2
3,6,2400,3,1,1000000,1013097,0,1,2
4,6,2701,3,1,1000000,1013097,0,1,2
...,...,...,...,...,...,...,...,...,...
1538866,8,300,4,56,1000000,1013097,104,1,1
1538867,8,500,4,56,1000000,1013097,80,2,1
1538868,8,400,4,56,1000000,1013097,102,5,1
1538869,8,100,4,56,1000000,1013097,50,2,1


# Read in csv and explore the data

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Import the nationwide data-it is presented in two files 
file_dir = '/Users/flatironschol/FIS-Projects/Module5/data/'
df_a = pd.read_csv('psam_husa.csv')
df_b = pd.read_csv('psam_husb.csv')
df = pd.concat((df_a, df_b), axis = 0, ignore_index = True) 

NameError: name 'pd' is not defined

In [97]:
tester = df
control = df

In [67]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@authors: 
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Function to drop weight columns
def drop_weights(data_frame):
    coltodrop = []
    for i in range(1, 81):
        coltodrop.append('WGTP' + str(i))
    return data_frame.drop(coltodrop, axis = 1)

# Function to drop allocation columns
def drop_allocations(data_frame):
    col = data_frame.columns
    coltodrop = []
    for c in col:
        if (c[0]== 'F') & (c[-1] == 'P') & \
           (c != 'FULFP') & (c != 'FULP') & (c != 'FINCP'): 
            coltodrop.append(c)
    return data_frame.drop(coltodrop, axis = 1)

In [21]:
# Function to recode the categorical features 
def recode(data_frame):
    # Binary categories
    dict_bin = {0: 1, 1: 2}
    bin_cols = ['HUGCL', 'NPP', 'NR', 'PSF', 'R18']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    dict_bin = {1: 2, 2: 1}
    bin_cols = ['BROADBND', 'COMPOTHX', 'DIALUP', 'HISPEED', 'LAPTOP', 'OTHSVCEX', \
                'SATELLITE', 'SMARTPHONE', 'TABLET', 'TEL', 'FS', 'BATH', 'REFR', \
                'SINK', 'STOV', 'KIT', 'RNTM']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    dict_bin = {0: 2, 1: 1}
    bin_cols = ['SRNT', 'SVAL']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    # Three categories
    dict_bin = {1: 2, 2: 1, 9:0}
    bin_cols = ['RWAT', 'RWATPR', 'PLM', 'PLMPRP', 'HOTWAT']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    dict_bin = {1: 2, 2: 1, 3: 3}
    bin_cols = ['ELEFP', 'FULFP', 'GASFP', 'WATFP']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    return data_frame
    

In [127]:
# Imputer
def imputer(data_frame):
    data_frame = data_frame.dropna(subset=['FS', 'HINCP'])
    data_frame.loc[:,'HINCP'] = data_frame.HINCP * \
                          data_frame.ADJINC / (10**6) 
    data_frame['WIF'].fillna(4, inplace=True)
    data_frame['WORKSTAT'].fillna(16, inplace=True)
    zero_list = ['VEH', 'BATH', 'REFR', 'SINK', 'STOV', 'KIT', 'BDSP', 'BLD', 'RWAT', 'RWATPR', 'PLM', 'PLMPRP', 'SRNT', 'SVAL', 'MV', 'TEN', 'CONP', 'ELEFP', 'ELEP', 'FULFP', 'FULP', 'GASFP', 'GASP', 'HFL']
    for col in zero_list:
        data_frame[col].fillna(0, inplace=True)
    data_frame = data_frame.drop(['ACR', 'AGS', 'FINCP', 'RESMODE', 'VALP', 'VACS'], axis = 1)

    return data_frame

In [128]:
tester = df
control = df

In [133]:
test2 = imputer(tester)
test2.ACR

AttributeError: 'DataFrame' object has no attribute 'ACR'

In [132]:
test2.RWATPR.unique()

array([0.])

In [121]:
tester.HINCP.unique()

array([    nan,  96400.,  59100., ..., 517130., 211660., 635600.])

In [69]:
tester = df[['HUGCL', 'NPP', 'NR', 'PSF', 'R18']]

In [75]:
tester.FULFP

AttributeError: 'DataFrame' object has no attribute 'FULFP'

In [73]:
dict_bin = {0: 1, 1: 2}
bin_cols = ['HUGCL', 'NPP', 'NR', 'PSF', 'R18']
for c in bin_cols:
    tester[c] = tester[c].map(dict_bin)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [72]:
tester.HUGCL

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
1538866    1.0
1538867    1.0
1538868    1.0
1538869    1.0
1538870    1.0
Name: HUGCL, Length: 1538871, dtype: float64

In [74]:
tester.HUGCL

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
1538866    2.0
1538867    2.0
1538868    2.0
1538869    2.0
1538870    2.0
Name: HUGCL, Length: 1538871, dtype: float64

In [149]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@authors: 
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Function to drop weight columns
def drop_weights(data_frame):
    coltodrop = []
    for i in range(1, 81):
        coltodrop.append('WGTP' + str(i))
    return data_frame.drop(coltodrop, axis = 1)

# Function to drop allocation columns
def drop_allocations(data_frame):
    col = data_frame.columns
    coltodrop = []
    for c in col:
        if (c[0]== 'F') & (c[-1] == 'P') & \
           (c != 'FULFP') & (c != 'FULP') & (c != 'FINCP'): 
            coltodrop.append(c)
    return data_frame.drop(coltodrop, axis = 1)

# Function to recode the categorical features 
def recode(data_frame):
    # Binary categories
    dict_bin = {0: 1, 1: 2}
    bin_cols = ['HUGCL', 'NPP', 'NR', 'PSF', 'R18']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    dict_bin = {1: 2, 2: 1}
    bin_cols = ['BROADBND', 'COMPOTHX', 'DIALUP', 'HISPEED', 'LAPTOP', 'OTHSVCEX', \
                'SATELLITE', 'SMARTPHONE', 'TABLET', 'TEL', 'BATH', 'REFR', \
                'SINK', 'STOV', 'KIT', 'RNTM']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    dict_bin = {0: 2, 1: 1}
    bin_cols = ['SRNT', 'SVAL']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    # Three categories
    dict_bin = {1: 2, 2: 1, 9:0}
    bin_cols = ['RWAT', 'RWATPR', 'PLM', 'PLMPRP', 'HOTWAT']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    dict_bin = {1: 2, 2: 1, 3: 3}
    bin_cols = ['ELEFP', 'FULFP', 'GASFP', 'WATFP']
    for c in bin_cols:
        data_frame[c] = data_frame[c].map(dict_bin)
    return data_frame
    
def recode_y(data_frame):
    dict_bin = {1: 2, 2: 1}
    data_frame = data_frame.map(dict_bin)
    
# Imputer
def imputer(data_frame):
    data_frame = data_frame.dropna(subset=['HINCP'])
    data_frame.loc[:,'HINCP'] = data_frame.HINCP * \
                          data_frame.ADJINC / (10**6) 
    data_frame['WIF'].fillna(4, inplace=True)
    data_frame['WORKSTAT'].fillna(16, inplace=True)
    zero_list = ['VEH', 'BATH', 'REFR', 'SINK', 'STOV', 'KIT', 'BDSP', 'BLD', 'RWAT', 'RWATPR', 'PLM', 'PLMPRP', 'SRNT', 'SVAL', 'MV', 'TEN', 'CONP', 'ELEFP', 'ELEP', 'FULFP', 'FULP', 'GASFP', 'GASP', 'HFL']
    for col in zero_list:
        data_frame[col].fillna(0, inplace=True)
    data_frame = data_frame.drop(['ACR', 'AGS', 'FINCP', 'RESMODE', 'VALP', 'VACS'], axis = 1)

    return data_frame    
    
# Import the nationwide data-it is presented in two files 
file_dir = '/Users/flatironschol/FIS-Projects/Module5/data/'
df_a = pd.read_csv('psam_husa.csv')
df_b = pd.read_csv('psam_husb.csv')
df_a = drop_weights(df_a)
df_a = drop_allocations(df_a)
df_b = drop_weights(df_b)
df_b = drop_allocations(df_b)
df = pd.concat((df_a, df_b), axis = 0) 
y = df.FS
X = df.drop('FS', axis = 1)
X = recode(X)
y = recode_y(y)
X = imputer(X)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, \
#                                                     stratify = y, random_state = 1007)

In [153]:
X.

(1257501, 92)

153475

In [16]:
 
# Import the nationwide data-it is presented in two files 
file_dir = '/Users/flatironschol/FIS-Projects/Module5/data/'
df_a = pd.read_csv('psam_husa.csv')
df_b = pd.read_csv('psam_husb.csv')
df_a = drop_weights(df_a)
df_a = drop_allocations(df_a)
df_b = drop_weights(df_b)
df_b = drop_allocations(df_b)
df = pd.concat((df_a, df_b), axis = 0) 
y = df.FS
X = df.drop('FS', axis = 1)
df = recode(df)
df = imputer(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, \
                                                    stratify = y, random_state = 1007)

In [14]:
imputer(dfus)

In [15]:
dftest.FS

AttributeError: 'NoneType' object has no attribute 'FS'

In [82]:
df.shape()

AttributeError: 'NoneType' object has no attribute 'shape'

In [64]:
testdf = pd.DataFrame(np.random.randn(10,3))

In [65]:
testdf.iloc[::2,0] = np.nan; testdf.iloc[::4,1] = np.nan; testdf.iloc[::3,2] = np.nan;

In [25]:
testdf

,0,1,2
0,NaN,NaN,NaN
1,-1.460030,1.338057,-1.249911
2,NaN,-0.076982,-0.632268
3,0.555522,-1.153359,NaN
4,NaN,NaN,0.901570
5,0.250432,0.425313,-1.105432
6,NaN,-0.334417,NaN
7,-0.599216,0.745599,0.001970
8,NaN,NaN,-0.352204
9,0.134864,1.484728,NaN


In [40]:
testdfdropped = testdf.dropna(subset = [2])

In [41]:
testdfdropped

,0,1,2
1,-1.460030,1.338057,-1.249911
2,NaN,-0.076982,-0.632268
4,NaN,NaN,0.901570
5,0.250432,0.425313,-1.105432
7,-0.599216,0.745599,0.001970
8,NaN,NaN,-0.352204


In [66]:
testdf[1].fillna('789789789', inplace=True)
testdf

,0,1,2
0,NaN,789789789,NaN
1,0.291206,-1.07383,-1.347451
2,NaN,0.0294676,-1.466109
3,1.254141,-0.0606889,NaN
4,NaN,789789789,-0.239185
5,-1.906831,1.07313,0.300592
6,NaN,0.082062,NaN
7,0.058171,-0.0242473,2.226617
8,NaN,789789789,-0.166204
9,1.119430,-1.05928,NaN


In [17]:
testdf.FS.unique()

array([ 2.,  1., nan])

In [18]:
testdfdropped = testdf.dropna(subset = ['FS'])

In [20]:
testdfdropped.FS.unique()

array([2., 1.])

In [21]:
testdf.shape

(1538871, 99)

In [22]:
testdfdropped.shape

(1410976, 99)

In [46]:
imptr(testdf.1)

SyntaxError: invalid syntax (<ipython-input-46-c1c4b96e601b>, line 1)